In [1]:
import pandas as pd
import ee
from collections import defaultdict
import numpy as np
import datetime
ee.Initialize()
import time

In [2]:
import time

# Parameters

In [3]:
# LOCATION OF CSV FILE
DATA = 'data/'

ALL_BANDS = True

# Download only pictures of rice and wheat
DOWNLOAD_ONLY_POSITIVE_EXAMPLE = False
DOWNLOAD_ONLY_NEGATIVE_EXAMPLE = False

# NUMBER OF FILES TO DOWNLOAD
# set to None to download all files
DOWNLOAD_NB_FILES = None

# RADIUS AROUND COORD IN METERS
# This is the number of meter around the point coordinate to include in the picture
RADIUS_AROUND = 400


# RANGES FOR BANDS
# Values tested on different location in nepal
# Hypothesis: Should be the same for all images
RANGE_MIN = 0
RANGE_MAX = 2000


# RANGE FOR DATES
# CSV shows one date per observation.
# However, we need to take several pictures to select ones without clouds
# We look for RANDE_DATE weeks around the date
RANDE_DATE = 1.5
NB_PICTURES_DATES = 12
RICE_TOI_END = '01/11/2018'
RICE_TOI_BEGIN = '01/06/2017'


# GOOGLE DRIVE FOLDER
GDRIVE_FOLDER = 'wfp-allbands-12months'


MAX_CLOUDS = 10

# Import CSV data a nd remove NaNs

In [4]:
df = pd.read_csv(DATA+'point_survey_v1.csv')
print(f'Len of input data: {len(df)}')

Len of input data: 1585


In [5]:
columns_suggestion = ['index', 'coord_obs_x', 'coord_obs_y', 'lc_code1', 'su_date', 'dist_m']
codes_monitored_crops = ['B101', 'B102', 'B103', 'B104']
df = df.reset_index()

In [6]:
df = df[columns_suggestion]

In [7]:
print(f'size df {len(df)}')
for col in columns_suggestion:
    print(f'Number of NaN for {col}: {df[col].isna().sum()}')

size df 1585
Number of NaN for index: 0
Number of NaN for coord_obs_x: 16
Number of NaN for coord_obs_y: 16
Number of NaN for lc_code1: 136
Number of NaN for su_date: 0
Number of NaN for dist_m: 13


In [8]:
df.dropna(inplace=True)

In [9]:
print(f'size df {len(df)}')
for col in columns_suggestion:
    print(f'Number of NaN for {col}: {df[col].isna().sum()}')

size df 1420
Number of NaN for index: 0
Number of NaN for coord_obs_x: 0
Number of NaN for coord_obs_y: 0
Number of NaN for lc_code1: 0
Number of NaN for su_date: 0
Number of NaN for dist_m: 0


In [10]:

name_map = defaultdict(lambda: 'other')

name_map['B104'] = 'wheat'
name_map['B101'] = 'rice'
name_map['B102'] = 'rice'
name_map['B103'] = 'rice'

df['type'] = df['lc_code1'].apply(lambda x: name_map[x])
if DOWNLOAD_ONLY_POSITIVE_EXAMPLE:
    df = df[df['type'] != 'other']
elif DOWNLOAD_ONLY_NEGATIVE_EXAMPLE:
    df = df[df['type'] == 'other']

In [11]:
old_size_df = len(df)
out_of_frame_images = len(df[df['dist_m'] > RADIUS_AROUND])
print(f"There are {out_of_frame_images} images too far, for the radius {RADIUS_AROUND}")
df = df[df['dist_m'] <= RADIUS_AROUND]
assert((out_of_frame_images + len(df)) == old_size_df)


There are 0 images too far, for the radius 400


In [12]:
print(f'Will download {len(df)} pictures')

Will download 1420 pictures


In [13]:
df['type'].value_counts()

other    1015
rice      387
wheat      18
Name: type, dtype: int64

# Google Earth Engine download


In [14]:
# Create list with a tuple per point: File name - x - y - date
coords = [(str(point[1][0])+'-'+str(point[1][3]), point[1][1], point[1][2], point[1][4], point[1][5]) for point in df.loc[:, ['index', 'coord_obs_x', 'coord_obs_y', 'lc_code1', 'su_date', 'type']].iterrows()]

In [15]:
# File name - x - y - date - type
coords[0]

('0-B401', 84.28586591, 27.42846051, '10/11/2016', 'other')

In [16]:
def stringify_date(date):
    #return f'{date.day}/{date.month}/{date.year}'
    
    return date.strftime("%d/%m/%Y")

def get_date(data_str):
    d, m, y = data_str.split('/')
    
    return datetime.date(int(y), int(m), int(d))

def generate_images_dates(image_collection, x, y, date, image_name, type_crop, debug=True, start_from=0):
  
        if start_from > 0:
            indice = int(image_name.split('-')[0])
            
            if indice < start_from:
                print(f'SKIP IMAGE {indice}')
                return -1


        begin = get_date(RICE_TOI_BEGIN)
        end = get_date(RICE_TOI_END)

        
        delta = ((end - begin) / NB_PICTURES_DATES)
        
        for i in range (NB_PICTURES_DATES):
            generate_image(image_collection, x, y, stringify_date(begin), image_name+'-'+str(begin), type_crop, debug)
            #generate_image(image_collection, x, y, date, image_name+'-'+str(get_date(date)), type_crop, debug)
            begin = begin+delta

  

In [17]:
# Generate a rectangle containing the circle (centered on the coordinate) with radius RADIUS_AROUND
def get_geometry_radius(geometry_point):
    coord = np.array(geometry_point.getInfo()['coordinates'][0])
    return ee.Geometry.Rectangle([coord[:, 0].min(), coord[:, 1].min(), coord[:, 0].max(), coord[:, 1].max()])

# Generate the dates around the observation date
def date_range_to_collect(input_date, type_crop, debug=False):
    d, m, y = input_date.split('/')
        
    target_date = datetime.date(int(y), int(m), int(d))
    delta = datetime.timedelta(weeks=RANDE_DATE)
    return target_date-delta, target_date+delta

def generate_image(image_collection, x, y, date, image_name, type_crop, debug=True):
    if debug: print(f'Working on {image_name}: ({x}, {y}) on {date}')
    geo = ee.Geometry.Point(x, y);
    radius = geo.buffer(RADIUS_AROUND);
    geometry_radius= get_geometry_radius(radius)
    
    spatialFiltered = image_collection.filterBounds(geo)

    date_range = date_range_to_collect(date, type_crop, debug)
    if debug: print('date range:' + str(date_range[0]) + str(date_range[1]))
    temporalFiltered = spatialFiltered.filterDate(str(date_range[0]), str(date_range[1]))

    # The darker pixels have the least clouds
    #least_clouds = temporalFiltered.min()
    least_clouds = temporalFiltered.sort('CLOUD_PIXEL_PERCENTAGE').first()
    #least_clouds = temporalFiltered.mosaic();
    #least_clouds = temporalFiltered.max()

#ar scene = ee.Image(sorted.first())

    if ALL_BANDS:
        testimg = least_clouds.select('B.+')
    else:
        testimg = least_clouds.visualize(bands=['B4', 'B3', 'B2'], min=RANGE_MIN, max=RANGE_MAX)
    
    #testimg = testimg.resample('bicubic')
    try:
        task = ee.batch.Export.image.toDrive(testimg, folder=GDRIVE_FOLDER, region=geometry_radius.getInfo()['coordinates'][0], description=image_name, scale=10)
        task.start()
        
    except:
            print('EXCEPTION DURING DOWNLOAD => TASK LIST SHOULD BE FULL. SLEEP FOR  2 hours')
            time.sleep(2*60*60)
            task = ee.batch.Export.image.toDrive(testimg, folder=GDRIVE_FOLDER, region=geometry_radius.getInfo()['coordinates'][0], description=image_name, scale=10)
            task.start()

In [18]:
df['type'].value_counts()

other    1015
rice      387
wheat      18
Name: type, dtype: int64

In [ ]:
dataset = ee.ImageCollection('COPERNICUS/S2').filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than',MAX_CLOUDS)

for i, point in enumerate(coords):
    if DOWNLOAD_NB_FILES is not None and i >= DOWNLOAD_NB_FILES:
        break
    generate_images_dates(dataset, point[1], point[2], point[3], point[0], point[4], start_from=0)

Working on 0-B401-2017-06-01: (84.28586591, 27.42846051) on 01/06/2017
date range:2017-05-222017-06-11
Working on 0-B401-2017-07-14: (84.28586591, 27.42846051) on 14/07/2017
date range:2017-07-042017-07-24
Working on 0-B401-2017-08-26: (84.28586591, 27.42846051) on 26/08/2017
date range:2017-08-162017-09-05
Working on 0-B401-2017-10-08: (84.28586591, 27.42846051) on 08/10/2017
date range:2017-09-282017-10-18
Working on 0-B401-2017-11-20: (84.28586591, 27.42846051) on 20/11/2017
date range:2017-11-102017-11-30
Working on 0-B401-2018-01-02: (84.28586591, 27.42846051) on 02/01/2018
date range:2017-12-232018-01-12
Working on 0-B401-2018-02-14: (84.28586591, 27.42846051) on 14/02/2018
date range:2018-02-042018-02-24
Working on 0-B401-2018-03-29: (84.28586591, 27.42846051) on 29/03/2018
date range:2018-03-192018-04-08
Working on 0-B401-2018-05-11: (84.28586591, 27.42846051) on 11/05/2018
date range:2018-05-012018-05-21
Working on 0-B401-2018-06-23: (84.28586591, 27.42846051) on 23/06/2018
da

date range:2018-05-012018-05-21
Working on 6-B102-2018-06-23: (84.40694692, 27.42943646) on 23/06/2018
date range:2018-06-132018-07-03
Working on 6-B102-2018-08-05: (84.40694692, 27.42943646) on 05/08/2018
date range:2018-07-262018-08-15
Working on 6-B102-2018-09-17: (84.40694692, 27.42943646) on 17/09/2018
date range:2018-09-072018-09-27
Working on 7-B102-2017-06-01: (84.40851276, 27.42944815) on 01/06/2017
date range:2017-05-222017-06-11
Working on 7-B102-2017-07-14: (84.40851276, 27.42944815) on 14/07/2017
date range:2017-07-042017-07-24
Working on 7-B102-2017-08-26: (84.40851276, 27.42944815) on 26/08/2017
date range:2017-08-162017-09-05
Working on 7-B102-2017-10-08: (84.40851276, 27.42944815) on 08/10/2017
date range:2017-09-282017-10-18
Working on 7-B102-2017-11-20: (84.40851276, 27.42944815) on 20/11/2017
date range:2017-11-102017-11-30
Working on 7-B102-2018-01-02: (84.40851276, 27.42944815) on 02/01/2018
date range:2017-12-232018-01-12
Working on 7-B102-2018-02-14: (84.4085127

Working on 15-E02-2017-11-20: (84.31741151, 27.4367469) on 20/11/2017
date range:2017-11-102017-11-30
Working on 15-E02-2018-01-02: (84.31741151, 27.4367469) on 02/01/2018
date range:2017-12-232018-01-12
Working on 15-E02-2018-02-14: (84.31741151, 27.4367469) on 14/02/2018
date range:2018-02-042018-02-24
Working on 15-E02-2018-03-29: (84.31741151, 27.4367469) on 29/03/2018
date range:2018-03-192018-04-08
Working on 15-E02-2018-05-11: (84.31741151, 27.4367469) on 11/05/2018
date range:2018-05-012018-05-21
Working on 15-E02-2018-06-23: (84.31741151, 27.4367469) on 23/06/2018
date range:2018-06-132018-07-03
Working on 15-E02-2018-08-05: (84.31741151, 27.4367469) on 05/08/2018
date range:2018-07-262018-08-15
Working on 15-E02-2018-09-17: (84.31741151, 27.4367469) on 17/09/2018
date range:2018-09-072018-09-27
Working on 16-F02-2017-06-01: (84.33603317, 27.43843619) on 01/06/2017
date range:2017-05-222017-06-11
Working on 16-F02-2017-07-14: (84.33603317, 27.43843619) on 14/07/2017
date range

date range:2018-06-132018-07-03
Working on 21-B102-2018-08-05: (84.30068409999998, 27.477949900000002) on 05/08/2018
date range:2018-07-262018-08-15
Working on 21-B102-2018-09-17: (84.30068409999998, 27.477949900000002) on 17/09/2018
date range:2018-09-072018-09-27
Working on 22-E02-2017-06-01: (84.34113539, 27.43730537) on 01/06/2017
date range:2017-05-222017-06-11
Working on 22-E02-2017-07-14: (84.34113539, 27.43730537) on 14/07/2017
date range:2017-07-042017-07-24
Working on 22-E02-2017-08-26: (84.34113539, 27.43730537) on 26/08/2017
date range:2017-08-162017-09-05
Working on 22-E02-2017-10-08: (84.34113539, 27.43730537) on 08/10/2017
date range:2017-09-282017-10-18
Working on 22-E02-2017-11-20: (84.34113539, 27.43730537) on 20/11/2017
date range:2017-11-102017-11-30
Working on 22-E02-2018-01-02: (84.34113539, 27.43730537) on 02/01/2018
date range:2017-12-232018-01-12
Working on 22-E02-2018-02-14: (84.34113539, 27.43730537) on 14/02/2018
date range:2018-02-042018-02-24
Working on 22

date range:2017-11-102017-11-30
Working on 28-B102-2018-01-02: (84.35766783, 27.44341668) on 02/01/2018
date range:2017-12-232018-01-12
Working on 28-B102-2018-02-14: (84.35766783, 27.44341668) on 14/02/2018
date range:2018-02-042018-02-24
Working on 28-B102-2018-03-29: (84.35766783, 27.44341668) on 29/03/2018
date range:2018-03-192018-04-08
Working on 28-B102-2018-05-11: (84.35766783, 27.44341668) on 11/05/2018
date range:2018-05-012018-05-21
Working on 28-B102-2018-06-23: (84.35766783, 27.44341668) on 23/06/2018
date range:2018-06-132018-07-03
Working on 28-B102-2018-08-05: (84.35766783, 27.44341668) on 05/08/2018
date range:2018-07-262018-08-15
Working on 28-B102-2018-09-17: (84.35766783, 27.44341668) on 17/09/2018
date range:2018-09-072018-09-27
Working on 29-B102-2017-06-01: (84.3562009, 27.44202807) on 01/06/2017
date range:2017-05-222017-06-11
Working on 29-B102-2017-07-14: (84.3562009, 27.44202807) on 14/07/2017
date range:2017-07-042017-07-24
Working on 29-B102-2017-08-26: (84

...

date range:2018-03-192018-04-08
Working on 840-F02-2018-05-11: (85.77431946, 27.39974812) on 11/05/2018
date range:2018-05-012018-05-21
Working on 840-F02-2018-06-23: (85.77431946, 27.39974812) on 23/06/2018
date range:2018-06-132018-07-03
Working on 840-F02-2018-08-05: (85.77431946, 27.39974812) on 05/08/2018
date range:2018-07-262018-08-15
Working on 840-F02-2018-09-17: (85.77431946, 27.39974812) on 17/09/2018
date range:2018-09-072018-09-27
Working on 841-G00-2017-06-01: (85.77284829, 27.39853409) on 01/06/2017
date range:2017-05-222017-06-11
Working on 841-G00-2017-07-14: (85.77284829, 27.39853409) on 14/07/2017
date range:2017-07-042017-07-24
Working on 841-G00-2017-08-26: (85.77284829, 27.39853409) on 26/08/2017
date range:2017-08-162017-09-05
Working on 841-G00-2017-10-08: (85.77284829, 27.39853409) on 08/10/2017
date range:2017-09-282017-10-18
Working on 841-G00-2017-11-20: (85.77284829, 27.39853409) on 20/11/2017
date range:2017-11-102017-11-30
Working on 841-G00-2018-01-02: (

date range:2017-08-162017-09-05
Working on 847-A21-2017-10-08: (85.6564021, 27.40778614) on 08/10/2017
date range:2017-09-282017-10-18
Working on 847-A21-2017-11-20: (85.6564021, 27.40778614) on 20/11/2017
date range:2017-11-102017-11-30
Working on 847-A21-2018-01-02: (85.6564021, 27.40778614) on 02/01/2018
date range:2017-12-232018-01-12
Working on 847-A21-2018-02-14: (85.6564021, 27.40778614) on 14/02/2018
date range:2018-02-042018-02-24
Working on 847-A21-2018-03-29: (85.6564021, 27.40778614) on 29/03/2018
date range:2018-03-192018-04-08
Working on 847-A21-2018-05-11: (85.6564021, 27.40778614) on 11/05/2018
date range:2018-05-012018-05-21
Working on 847-A21-2018-06-23: (85.6564021, 27.40778614) on 23/06/2018
date range:2018-06-132018-07-03
Working on 847-A21-2018-08-05: (85.6564021, 27.40778614) on 05/08/2018
date range:2018-07-262018-08-15
Working on 847-A21-2018-09-17: (85.6564021, 27.40778614) on 17/09/2018
date range:2018-09-072018-09-27
Working on 848-B107-2017-06-01: (85.65794

date range:2018-06-132018-07-03
Working on 853-E01-2018-08-05: (85.74795345, 27.41166013) on 05/08/2018
date range:2018-07-262018-08-15
Working on 853-E01-2018-09-17: (85.74795345, 27.41166013) on 17/09/2018
date range:2018-09-072018-09-27
Working on 854-B106-2017-06-01: (85.74893678, 27.41187761) on 01/06/2017
date range:2017-05-222017-06-11
Working on 854-B106-2017-07-14: (85.74893678, 27.41187761) on 14/07/2017
date range:2017-07-042017-07-24
Working on 854-B106-2017-08-26: (85.74893678, 27.41187761) on 26/08/2017
date range:2017-08-162017-09-05
Working on 854-B106-2017-10-08: (85.74893678, 27.41187761) on 08/10/2017
date range:2017-09-282017-10-18
Working on 854-B106-2017-11-20: (85.74893678, 27.41187761) on 20/11/2017
date range:2017-11-102017-11-30
Working on 854-B106-2018-01-02: (85.74893678, 27.41187761) on 02/01/2018
date range:2017-12-232018-01-12
Working on 854-B106-2018-02-14: (85.74893678, 27.41187761) on 14/02/2018
date range:2018-02-042018-02-24
Working on 854-B106-2018-

Working on 860-E01-2017-11-20: (85.7234329, 27.43476246) on 20/11/2017
date range:2017-11-102017-11-30
Working on 860-E01-2018-01-02: (85.7234329, 27.43476246) on 02/01/2018
date range:2017-12-232018-01-12
Working on 860-E01-2018-02-14: (85.7234329, 27.43476246) on 14/02/2018
date range:2018-02-042018-02-24
Working on 860-E01-2018-03-29: (85.7234329, 27.43476246) on 29/03/2018
date range:2018-03-192018-04-08
Working on 860-E01-2018-05-11: (85.7234329, 27.43476246) on 11/05/2018
date range:2018-05-012018-05-21
Working on 860-E01-2018-06-23: (85.7234329, 27.43476246) on 23/06/2018
date range:2018-06-132018-07-03
Working on 860-E01-2018-08-05: (85.7234329, 27.43476246) on 05/08/2018
date range:2018-07-262018-08-15
Working on 860-E01-2018-09-17: (85.7234329, 27.43476246) on 17/09/2018
date range:2018-09-072018-09-27
Working on 861-D10-2017-06-01: (85.72376264, 27.43418471) on 01/06/2017
date range:2017-05-222017-06-11
Working on 861-D10-2017-07-14: (85.72376264, 27.43418471) on 14/07/2017


Working on 866-B714-2018-09-17: (85.71321427, 27.438556199999997) on 17/09/2018
date range:2018-09-072018-09-27
Working on 867-B714-2017-06-01: (85.72315458, 27.43517789) on 01/06/2017
date range:2017-05-222017-06-11
Working on 867-B714-2017-07-14: (85.72315458, 27.43517789) on 14/07/2017
date range:2017-07-042017-07-24
Working on 867-B714-2017-08-26: (85.72315458, 27.43517789) on 26/08/2017
date range:2017-08-162017-09-05
Working on 867-B714-2017-10-08: (85.72315458, 27.43517789) on 08/10/2017
date range:2017-09-282017-10-18
Working on 867-B714-2017-11-20: (85.72315458, 27.43517789) on 20/11/2017
date range:2017-11-102017-11-30
Working on 867-B714-2018-01-02: (85.72315458, 27.43517789) on 02/01/2018
date range:2017-12-232018-01-12
Working on 867-B714-2018-02-14: (85.72315458, 27.43517789) on 14/02/2018
date range:2018-02-042018-02-24
Working on 867-B714-2018-03-29: (85.72315458, 27.43517789) on 29/03/2018
date range:2018-03-192018-04-08
Working on 867-B714-2018-05-11: (85.72315458, 27